## Create tools

In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [ ]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "The get_weather function retrieves the current temperature in Celsius for a given city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "City name to get the current temperature for",
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

## Testing the function

In [4]:
from openai import OpenAI
client = OpenAI()

In [11]:
import chat_assistant

In [13]:
tools = chat_assistant.Tools()
tools.get_tools()

[]

In [15]:
tools.add_tool(get_weather, get_weather_tool)
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'The get_weather function retrieves the current temperature for a given city. ',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'City name to get the current temperature for'}},
   'required': ['city'],
   'additionalProperties': False}}]

In [ ]:
developer_prompt = """
You are a helpful assistant that can provide the current temperature for any city.

Respond naturally, and include the temperature in your message when available.

Always ask a follow-up question to continue the conversation.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [21]:
chat.run()

Chat ended.


### Adding another tool

In [22]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [25]:
set_weather_tool = {
        "type": "function",
        "name": "set_weather",
        "description": "The set_weather function allows you to set the current temperature for a given city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "City name to set the current temperature for",
                },
                "temp": {
                    "type": "number",
                    "description": "Temperature in Celsius to set for the city",
                }
            },
            "required": ["city", "temp"],
            "additionalProperties": False
        }
    }

In [24]:
tools.add_tool(set_weather, set_weather_tool())
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'The get_weather function retrieves the current temperature for a given city. ',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'City name to get the current temperature for'}},
   'required': ['city'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'set_weather',
  'description': 'The set_weather function allows you to set the current temperature for a given city.',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'City name to set the current temperature for'},
    'temp': {'type': 'number',
     'description': 'Temperature in Celsius to set for the city'}},
   'required': ['city', 'temp'],
   'additionalProperties': False}}]

In [27]:
chat.run()

Chat ended.


## MCP

In [31]:
from fastmcp import Client

In [37]:
import weather_mcp
import json

In [39]:
async def main():
    async with Client(weather_mcp.mcp) as mcp_client:
        tools = await mcp_client.list_tools()
        print(tools)

await main()

[Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Te

## Test MCP client

In [44]:
import mcp_client

In [45]:
class MCPTools:
    def __init__(self, mcp_client):
        self.mcp_client = mcp_client
        self.tools = None
    
    def get_tools(self):
        if self.tools is None:
            mcp_tools = self.mcp_client.get_tools()
            self.tools = convert_tools_list(mcp_tools)
        return self.tools

    def function_call(self, tool_call_response):
        function_name = tool_call_response.name
        arguments = json.loads(tool_call_response.arguments)

        result = self.mcp_client.call_tool(function_name, arguments)

        return {
            "type": "function_call_output",
            "call_id": tool_call_response.call_id,
            "output": json.dumps(result, indent=2),
        }

In [47]:
our_mcp_client = mcp_client.MCPClient(["python", "weather_mcp.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

mcp_tools = mcp_client.MCPTools(mcp_client=our_mcp_client)


developer_prompt = """
You help users find out the weather in their cities. 
If they didn't specify a city, ask them. Make sure we always use a city.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=mcp_tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

chat.run()

Started server with command: python weather_mcp.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully
Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'London'}


Calling tool 'set_weather' with arguments: {'city': 'London', 'temp': 7}


Calling tool 'get_weather' with arguments: {'city': 'London'}


Chat ended.
